# 🔍 MMTD Attention 기반 해석가능성 분석 데모

이 노트북은 MMTD 모델의 Attention 기반 해석가능성 분석 시스템을 시연합니다.

## 🎯 주요 기능
- **텍스트 Attention**: 어떤 단어가 중요한지 분석
- **이미지 Attention**: 어떤 이미지 영역이 중요한지 분석
- **Cross-Modal Attention**: 텍스트와 이미지 간 상호작용 분석
- **종합 해석**: 예측 결과에 대한 완전한 설명 제공

---

In [ ]:
# 필요한 라이브러리 설치
import sys
import os
sys.path.append('..')

import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# 커스텀 모듈 import
from src.analysis.attention_analyzer import AttentionAnalyzer
from src.analysis.attention_visualizer import AttentionVisualizer
from src.models.interpretable_mmtd import InterpretableMMTD
from transformers import AutoTokenizer

print("🚀 라이브러리 로딩 완료!")
print(f"🖥️ 사용 가능한 디바이스: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

## 📥 모델 및 분석 도구 로딩

In [ ]:
# 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = '../checkpoints/best_interpretable_mmtd.ckpt'  # 실제 모델 경로로 수정
tokenizer_name = 'bert-base-uncased'

print(f"🔧 설정:")
print(f"  - 디바이스: {device}")
print(f"  - 모델 경로: {model_path}")
print(f"  - 토크나이저: {tokenizer_name}")

# 모델 로딩
try:
    model = InterpretableMMTD.load_from_checkpoint(
        model_path,
        map_location=device
    )
    model.to(device)
    model.eval()
    print("✅ 모델 로딩 성공!")
except Exception as e:
    print(f"❌ 모델 로딩 실패: {e}")
    print("💡 모델 경로를 확인하거나 사전 훈련된 모델을 사용해주세요.")

# 토크나이저 로딩
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
print("✅ 토크나이저 로딩 성공!")

# 분석 도구 초기화
analyzer = AttentionAnalyzer(model, tokenizer, device)
visualizer = AttentionVisualizer()
print("✅ 분석 도구 초기화 완료!")

## 📝 예시 데이터 준비

In [ ]:
# 예시 스팸 이메일 텍스트
spam_text = """
URGENT! You have WON $1,000,000 in our EXCLUSIVE lottery! 
Click here NOW to claim your FREE prize! 
Limited time offer - expires TODAY!
Call 1-800-WIN-CASH immediately!
"""

# 예시 정상 이메일 텍스트
ham_text = """
Hi John,

I hope you're doing well. I wanted to follow up on our meeting yesterday 
about the project timeline. Could you please send me the updated schedule 
when you have a chance?

Thanks,
Sarah
"""

print("📝 예시 텍스트 준비 완료!")
print(f"스팸 텍스트 길이: {len(spam_text)} 글자")
print(f"정상 텍스트 길이: {len(ham_text)} 글자")

# 예시 이미지 생성 (실제로는 데이터셋에서 로딩)
# 더미 이미지 (224x224 RGB)
dummy_image = torch.randn(3, 224, 224)
print(f"🖼️ 예시 이미지 shape: {dummy_image.shape}")

print("\n⚠️ 실제 사용 시에는 데이터셋에서 실제 이미지를 로딩해주세요!")

## 🔍 단일 샘플 Attention 분석

In [ ]:
# 스팸 텍스트 분석
print("🔍 스팸 텍스트 Attention 분석 중...")

try:
    spam_explanation = analyzer.explain_prediction(
        text=spam_text,
        image=dummy_image,
        return_attention_maps=True
    )
    
    print("✅ 분석 완료!")
    
    # 예측 결과 출력
    pred = spam_explanation['prediction']
    print(f"\n📊 예측 결과:")
    print(f"  • 예측 라벨: {pred['label']}")
    print(f"  • 예측 점수: {pred['score']:.4f}")
    print(f"  • 신뢰도: {pred['confidence']:.4f}")
    
    # 상위 중요 토큰 출력
    important_tokens = spam_explanation['text_analysis']['important_tokens'][:5]
    print(f"\n📝 가장 중요한 텍스트 토큰 (Top 5):")
    for i, token in enumerate(important_tokens, 1):
        print(f"  {i}. '{token['token']}' - 중요도: {token['combined_importance']:.4f}")
    
    # 모달리티 균형
    cross_modal = spam_explanation['cross_modal_analysis']
    print(f"\n⚖️ 모달리티 균형:")
    print(f"  • 텍스트→이미지: {cross_modal['text_to_image_strength']:.4f}")
    print(f"  • 이미지→텍스트: {cross_modal['image_to_text_strength']:.4f}")
    print(f"  • 균형도: {cross_modal['modality_balance']:.4f} (0=텍스트 중심, 1=이미지 중심)")
    
except Exception as e:
    print(f"❌ 분석 실패: {e}")
    spam_explanation = None

## 📊 텍스트 Attention 시각화

In [ ]:
if spam_explanation is not None:
    # 텍스트 attention 시각화
    text_fig = visualizer.visualize_text_attention(
        tokens=spam_explanation['text_analysis']['tokens'],
        token_importance=spam_explanation['text_analysis']['important_tokens'],
        title="스팸 텍스트 Attention 분석"
    )
    
    plt.show()
    print("📊 텍스트 attention 시각화 완료!")
else:
    print("❌ 분석 결과가 없어 시각화를 건너뜁니다.")

## 🖼️ 이미지 Attention 시각화

In [ ]:
if spam_explanation is not None:
    # 이미지 attention 시각화
    image_fig = visualizer.visualize_image_attention(
        image=dummy_image,
        patch_importance=spam_explanation['image_analysis']['important_patches'],
        title="스팸 이미지 Attention 분석"
    )
    
    plt.show()
    print("🖼️ 이미지 attention 시각화 완료!")
else:
    print("❌ 분석 결과가 없어 시각화를 건너뜁니다.")

## 🔄 Cross-Modal Attention 시각화

In [ ]:
if spam_explanation is not None and 'attention_maps' in spam_explanation:
    # Cross-modal attention 시각화
    cross_modal_fig = visualizer.visualize_cross_modal_attention(
        cross_modal_attention=spam_explanation['attention_maps']['cross_modal_attention'],
        tokens=spam_explanation['text_analysis']['tokens'],
        title="스팸 Cross-Modal Attention 분석"
    )
    
    plt.show()
    print("🔄 Cross-modal attention 시각화 완료!")
else:
    print("❌ Cross-modal attention 데이터가 없어 시각화를 건너뜁니다.")

## 🎯 종합 분석 시각화

In [ ]:
if spam_explanation is not None:
    # 종합 분석 시각화
    comprehensive_fig = visualizer.visualize_comprehensive_explanation(
        explanation=spam_explanation,
        image=dummy_image,
        title="스팸 이메일 종합 Attention 분석"
    )
    
    plt.show()
    print("🎯 종합 분석 시각화 완료!")
else:
    print("❌ 분석 결과가 없어 시각화를 건너뜁니다.")

## 🔄 정상 이메일 비교 분석

In [ ]:
# 정상 이메일 분석
print("🔍 정상 이메일 Attention 분석 중...")

try:
    ham_explanation = analyzer.explain_prediction(
        text=ham_text,
        image=dummy_image,
        return_attention_maps=True
    )
    
    print("✅ 정상 이메일 분석 완료!")
    
    # 스팸 vs 정상 비교
    if spam_explanation is not None:
        print("\n📊 스팸 vs 정상 이메일 비교:")
        print("="*50)
        
        print(f"스팸 이메일:")
        print(f"  • 예측: {spam_explanation['prediction']['label']} ({spam_explanation['prediction']['score']:.4f})")
        print(f"  • 모달리티 균형: {spam_explanation['cross_modal_analysis']['modality_balance']:.4f}")
        
        print(f"\n정상 이메일:")
        print(f"  • 예측: {ham_explanation['prediction']['label']} ({ham_explanation['prediction']['score']:.4f})")
        print(f"  • 모달리티 균형: {ham_explanation['cross_modal_analysis']['modality_balance']:.4f}")
        
        # 중요 토큰 비교
        spam_tokens = [t['token'] for t in spam_explanation['text_analysis']['important_tokens'][:3]]
        ham_tokens = [t['token'] for t in ham_explanation['text_analysis']['important_tokens'][:3]]
        
        print(f"\n📝 중요 토큰 비교:")
        print(f"스팸: {spam_tokens}")
        print(f"정상: {ham_tokens}")
    
except Exception as e:
    print(f"❌ 정상 이메일 분석 실패: {e}")
    ham_explanation = None

## 📋 결과 저장

In [ ]:
# 결과 저장
output_dir = '../outputs/demo_results'
os.makedirs(output_dir, exist_ok=True)

if spam_explanation is not None:
    # 스팸 분석 결과 저장
    analyzer.save_explanation(
        spam_explanation,
        f'{output_dir}/spam_explanation.json',
        include_attention_maps=False
    )
    print("💾 스팸 분석 결과 저장 완료!")

if ham_explanation is not None:
    # 정상 분석 결과 저장
    analyzer.save_explanation(
        ham_explanation,
        f'{output_dir}/ham_explanation.json',
        include_attention_maps=False
    )
    print("💾 정상 분석 결과 저장 완료!")

print(f"\n📁 결과 저장 위치: {output_dir}")

## 🎉 데모 완료!

### 🔍 **분석된 내용**
1. **텍스트 Attention**: 어떤 단어가 스팸/정상 판단에 중요한지
2. **이미지 Attention**: 어떤 이미지 영역이 중요한지
3. **Cross-Modal Attention**: 텍스트와 이미지가 어떻게 상호작용하는지
4. **모달리티 균형**: 텍스트 vs 이미지 기여도

### 🚀 **다음 단계**
- 실제 데이터셋으로 배치 분석 실행: `scripts/attention_analysis_experiment.py`
- 더 많은 샘플로 패턴 분석
- 오류 사례 심층 분석
- 모델 개선점 도출

### 📊 **핵심 장점**
- ✅ **완전한 투명성**: 모든 예측에 대한 명확한 근거 제공
- ✅ **다중모달 해석**: 텍스트와 이미지 모두 분석
- ✅ **직관적 시각화**: 비전문가도 이해 가능
- ✅ **실용적 활용**: 실제 스팸 필터링 시스템에 적용 가능

---
*🔬 이것이 바로 "진짜 해석가능한" AI입니다!*